# 03 — LSTM Forecasting

In [ ]:
!pip -q install -r ../requirements.txt

In [ ]:
import pandas as pd
import torch
from src.data_loader import load_series, time_split
from src.lstm_model import train_lstm, forecast_lstm
from src.evaluation import compute_metrics
from src.visualization import plot_forecast

df = load_series('../data/raw/daily_energy_synthetic.csv')
train, val, test = time_split(df, test_days=90, val_days=60)

hist = pd.concat([train, val])
H = len(test)
y_true = test['value'].to_numpy(float)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, norm = train_lstm(train['value'].to_numpy(float), val['value'].to_numpy(float), lookback=30,
                         hidden_size=64, num_layers=1, epochs=15, batch_size=64, lr=1e-3, device=device)

lstm = forecast_lstm(model, hist['value'].to_numpy(float), horizon=H, lookback=30, norm_stats=norm, device=device)
print('LSTM:', compute_metrics(y_true, lstm))

plot_forecast(test['date'].to_numpy(), y_true, {'LSTM': lstm})